<a href="https://colab.research.google.com/github/aigonna/ML_Skills/blob/main/4_optuna_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cp -r /content/drive/Shareddrives/mm/kaggles/tatanic/data /content/

In [ ]:
pip install optuna

     |████████████████████████████████| 302 kB 7.8 MB/s 
     |████████████████████████████████| 80 kB 11.6 MB/s 
     |████████████████████████████████| 164 kB 90.3 MB/s 
     |████████████████████████████████| 75 kB 5.9 MB/s 
     |████████████████████████████████| 111 kB 89.0 MB/s 
     |████████████████████████████████| 141 kB 83.2 MB/s 
     |████████████████████████████████| 49 kB 8.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=55689d8698194ccbe9bb25fe672a82433845c61409a9fdc28dc37ae57f9931bc
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, KFold

import xgboost as xgb
import optuna
from optuna.samplers import TPESampler

from pathlib import Path


In [ ]:
data_dir = Path("/content/data")

train_data = pd.read_csv(data_dir/'train.csv')
test_data = pd.read_csv(data_dir/'test.csv')
sample = pd.read_csv(data_dir/'gender_submission.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 1.预处理数据

In [ ]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)  # 搜索字母后接.像Mr.这种敬称
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""


def create_extra_features(data):
    data['Ticket_type'] = data['Ticket'].map(lambda x: x[0:3])  # Ticket_type只要Ticket前3字符
    data['Name_Words_Count'] = data['Name'].map(lambda x: len(x.split()))  # name 词数
    # 如果不nan就是有Cabin,转换为1,0int型数据
    data['Has_Cabin'] = data["Cabin"].map(lambda x: 1 - int(type(x) == float))
    data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

    
    data['Fare'] = data['Fare'].fillna(data['Fare'].mean()).astype('int')
    data['Age'] = data['Age'].fillna(data['Age'].mean()).astype('int')

    data['Title'] = data['Name'].apply(get_title).replace(
        ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    data['Title'] = data['Title'].replace('Mlle', 'Miss')
    data['Title'] = data['Title'].replace('Ms', 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')
    data['Title'] = data['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}).fillna(0)
    data['Sex'] = data['Sex'].map({'female': 1, 'male':0}).astype(int)
    data['Embarked'] = data['Embarked'].map({"C":0,"Q":1,"S":2}).fillna(0).astype(int)

    return data

In [ ]:
train_data = create_extra_features(train_data)
test_data = create_extra_features(test_data)

In [ ]:
train_data.drop(['PassengerId', 'Name', 'Ticket', 'Ticket_type', 'Cabin'], axis=1, inplace=True)
test_data.drop(['PassengerId', 'Name', 'Ticket', 'Ticket_type', 'Cabin'], axis=1, inplace=True)
X = train_data.drop(['Survived'], axis=1, inplace=False)
y = train_data['Survived']

In [ ]:
def f1_metric(y_true, y_pred):
    return f1_score(y_true, (y_pred > 0.5).astype(int))


In [ ]:
X.head(), test_data.shape

(   Pclass  Sex  Age  SibSp  ...  Name_Words_Count  Has_Cabin  FamilySize  Title
 0       3    0   22      1  ...                 4          0           2      1
 1       1    1   38      1  ...                 7          1           2      3
 2       3    1   26      0  ...                 3          0           1      2
 3       1    1   35      1  ...                 7          1           2      3
 4       3    0   35      0  ...                 4          0           1      1
 
 [5 rows x 11 columns], (418, 11))

In [ ]:
def objective(trial):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                test_size=0.2,
                                stratify=y,
                                random_state=2020)
    params = {
        'n_estimators': trial.suggest_int('n_estimator', 100, 2500, 200),
        'eta': trial.suggest_loguniform('eta', 1e-6, 1e-3), #从1e-6～1e-3的log uniform分布里选
        'max_depth': trial.suggest_int('max_depth', 5, 25, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 35, 2),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_by_tree', 0.5, 0.9, 0.1),
        'reg_lambda': trial.suggest_discrete_uniform('reg_lambda', 0.001, 20.001, 0.1),
        'reg_alpha': trial.suggest_discrete_uniform('reg_alpha', 0.001, 20.001, 0.1),
        'gamma': trial.suggest_loguniform('gamma', 1e-5, 10),
    }
    classifer = xgb.XGBClassifier(
        objective='binary:logistic',
        tree_method='gpu_hist',
        n_jobs=-1,
        **params,
    )

    classifer.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],\
                  eval_metric="error", verbose=False)

    return f1_metric(y_valid, classifer.predict(X_valid))

In [ ]:
training_time = 1*60*60

study = optuna.create_study(direction='maximize', study_name='XGBClassifier', sampler=TPESampler())
study.optimize(objective, timeout=training_time)

[I 2021-08-29 04:52:05,734] A new study created in memory with name: XGBClassifier
[I 2021-08-29 04:52:12,506] Trial 0 finished with value: 0.7669172932330827 and parameters: {'n_estimator': 2500, 'eta': 5.985414607869781e-06, 'max_depth': 17, 'min_child_weight': 19, 'subsample': 0.6, 'colsample_by_tree': 0.5, 'reg_lambda': 3.201, 'reg_alpha': 3.501, 'gamma': 4.744828656690232e-05}. Best is trial 0 with value: 0.7669172932330827.
[I 2021-08-29 04:52:13,916] Trial 1 finished with value: 0.7647058823529411 and parameters: {'n_estimator': 700, 'eta': 0.0004946553084328892, 'max_depth': 5, 'min_child_weight': 21, 'subsample': 0.7, 'colsample_by_tree': 0.5, 'reg_lambda': 11.901, 'reg_alpha': 2.201, 'gamma': 3.486498758330956e-05}. Best is trial 0 with value: 0.7669172932330827.
[I 2021-08-29 04:52:18,387] Trial 2 finished with value: 0.7777777777777778 and parameters: {'n_estimator': 2300, 'eta': 1.4475851872725162e-06, 'max_depth': 5, 'min_child_weight': 17, 'subsample': 0.8, 'colsample_by

In [ ]:
print('完成试验次数：', len(study.trials))
trail = study.best_trial
print('最佳试验:', trail)
print('\tValue: {:.4f}'.format(trail.value))
print("具体参数如下:")
for k, v in trail.params.items():
    print('\t\t{}:{}'.format(k, v))

完成试验次数： 131
最佳试验: FrozenTrial(number=83, values=[0.7910447761194029], datetime_start=datetime.datetime(2021, 8, 29, 5, 15, 46, 330979), datetime_complete=datetime.datetime(2021, 8, 29, 5, 15, 59, 93714), params={'n_estimator': 700, 'eta': 0.000740769797441306, 'max_depth': 21, 'min_child_weight': 27, 'subsample': 0.9, 'colsample_by_tree': 0.9, 'reg_lambda': 11.101, 'reg_alpha': 3.4010000000000002, 'gamma': 2.2646419765313437e-05}, distributions={'n_estimator': IntUniformDistribution(high=2500, low=100, step=200), 'eta': LogUniformDistribution(high=0.001, low=1e-06), 'max_depth': IntUniformDistribution(high=25, low=5, step=1), 'min_child_weight': IntUniformDistribution(high=35, low=1, step=2), 'subsample': DiscreteUniformDistribution(high=0.9, low=0.5, q=0.1), 'colsample_by_tree': DiscreteUniformDistribution(high=0.9, low=0.5, q=0.1), 'reg_lambda': DiscreteUniformDistribution(high=20.001, low=0.001, q=0.1), 'reg_alpha': DiscreteUniformDistribution(high=20.001, low=0.001, q=0.1), 'gamma'

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:

optuna.visualization.plot_param_importances(study)

In [ ]:
xgb_params = trail.params
xgb_params['objective'] = 'binary:logistic'
xgb_params['tree_method'] = 'gpu_hist'
xgb_params['predictor'] = 'gpu_predictor'
xgb_params['n_jobs'] = -1

In [ ]:
n_splits = 10
test_preds = None
kf_f1 = [] #每个分fold的验证集f1列表

for fold, (train_idx, valid_idx) in enumerate(KFold(n_splits=n_splits, shuffle=True).split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[valid_idx], y.iloc[valid_idx]

    model = xgb.XGBClassifier(**xgb_params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='error', verbose=False)

    valid_pred = model.predict(X_val)
    f1 = f1_metric(y_val, valid_pred)
    print(f'kFold {fold + 1} / {n_splits} f1:{f1:.4f}')
    kf_f1.append(f1)

    if test_preds is None:
        test_preds = model.predict(test_data)
    else:
        test_preds += model.predict(test_data)


kFold 1 / 10 f1:0.7241
kFold 2 / 10 f1:0.7595
kFold 3 / 10 f1:0.8065
kFold 4 / 10 f1:0.8219
kFold 5 / 10 f1:0.7586
kFold 6 / 10 f1:0.6301
kFold 7 / 10 f1:0.8116
kFold 8 / 10 f1:0.7941
kFold 9 / 10 f1:0.6800
kFold 10 / 10 f1:0.7879


In [ ]:
test_preds = test_preds/n_splits
print(f'Average KFold f1 score :{np.mean(np.array(kf_f1)):.5f}')

Average KFold f1 score :0.75743


In [ ]:
test_preds

array([0. , 0. , 0. , 0. , 1. , 0. , 1. , 0. , 1. , 0. , 0. , 0. , 1. ,
       0. , 1. , 1. , 0. , 0. , 1. , 1. , 0. , 0. , 1. , 0. , 1. , 0. ,
       1. , 0. , 0. , 0. , 0. , 0. , 0.4, 0.3, 0.7, 0. , 1. , 0.9, 0. ,
       0. , 0. , 0. , 0. , 1. , 1. , 0. , 0. , 0. , 1. , 1. , 0.1, 0. ,
       1. , 1. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 1. , 1. ,
       1. , 1. , 0. , 0.4, 1. , 1. , 0. , 1. , 0.4, 1. , 0.7, 0. , 1. ,
       0. , 1. , 0.1, 0. , 0. , 0. , 0. , 0. , 1. , 1. , 1. , 1. , 1. ,
       0. , 1. , 0. , 0. , 0. , 1. , 0. , 1. , 0. , 1. , 0. , 0. , 0. ,
       1. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. , 1. , 1. , 0. , 0. ,
       1. , 0.3, 1. , 1. , 0. , 1. , 0. , 0. , 1. , 0. , 1. , 0. , 0. ,
       0. , 1. , 0.1, 0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0. , 1. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0.8, 1. , 0. , 0. , 1. , 0. , 0. ,
       1. , 1. , 0. , 1. , 1. , 0.2, 1. , 0. , 0.1, 1. , 0. , 0. , 1. ,
       1. , 0. , 0. , 0. , 0. , 0. , 1. , 1. , 0. , 1. , 1. , 0.

In [ ]:
sample['Survived'] = (test_preds > 0.5).astype(int)
sample.to_csv('/content/xgb_optuna.csv', index=False)

In [ ]:
!rm -f /root/.kaggle/kaggle.json
!mkdir /root/.kaggle/
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c titanic -f /content/xgb_optuna.csv -m "optuna_xgb"

100% 2.77k/2.77k [00:01<00:00, 1.61kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

In [ ]:
##0.77990